In [1]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import numpy as np
import requests
from IPython.display import display, Markdown

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [2]:
def add_repo_root_path():
    import os
    import sys
    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if repo_root not in sys.path:
        sys.path.append(repo_root)
        
add_repo_root_path()
from src import generate_knowledge
from src import create_rag_db
from src import llm_chain_tools
from src.enhanced_retriever import EnhancedRetriever

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

In [4]:
generate_knowledge.add_repo_root_path()
import openai_setup

OPENAI_API_KEY = openai_setup.conf['key']
OPENAI_PROJECT = openai_setup.conf['project']
OPENAI_ORGANIZATION = openai_setup.conf['organization']
DEFAULT_LLM_MODEL = "gpt-4o-mini"
CHROMADB_DIRECTORY = '../chromadb'
COLLECTION_NAME = "my_chromadb" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ['OPENAI_MODEL_NAME'] = DEFAULT_LLM_MODEL

In [5]:
langchain_openai_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
langchain_openai_llm = ChatOpenAI(model=DEFAULT_LLM_MODEL, temperature=0.1, openai_api_key=OPENAI_API_KEY, openai_organization = OPENAI_ORGANIZATION)

In [24]:
from langchain_openai import ChatOpenAI

loaded_vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=CHROMADB_DIRECTORY,
    embedding_function=langchain_openai_embeddings
)

_, repo_name = generate_knowledge.extract_owner_and_repo('https://github.com/dbt-labs/jaffle-shop')
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')
dbt_repo_knowledge_df = create_rag_db.merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df)

import nest_asyncio
nest_asyncio.apply()

import importlib
import src.llm_agents_flow
importlib.reload(src.llm_agents_flow)
from src.llm_agents_flow import dbtChatFlow

files = {
    'agents': '../config/agents.yml',
    'tasks': '../config/tasks.yml'
}

In [25]:
flow = dbtChatFlow(files)
flow.plot()

user_input = "I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it "
result = flow.kickoff(inputs={"request": user_input, "dbt_repo_knowledge_df": dbt_repo_knowledge_df, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})
display(Markdown(f"<div style='font-size: 18px;'><b>User input:</b> <i>{user_input}</i></div><hr>"))
display(Markdown(result.raw))

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


Plot saved as crewai_flow.html
# Agent: Identify if the user's request explicitly mentions a specific model for retrieving information or implementing changes.
## Task: Verify if the request explicitly mentions a model that requires information retrieval or changes. Request: I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it  Current dbt lineage of the dbt project:                model_name                source  \
0           stg_customers  [ecom.raw_customers]   
1           stg_locations     [ecom.raw_stores]   
2         stg_order_items      [ecom.raw_items]   
3              stg_orders     [ecom.raw_orders]   
4            stg_products   [ecom.raw_products]   
5            stg_supplies   [ecom.raw_supplies]   
6               customers                    []   
7               locations                    []   
8   met

CrewOutput(raw='# User Request \nThe user requests to add a new column \'overdue\' to the model `orders` that derives from the `raw_orders` source, without requiring any calculations since this column is already available in the raw data. The user also wants this new column to be accessible in the related `customers` model.\n\n# Validation of Proposed Changes \n\n- For the main model: \n  - **Proposed Changes:**\n    1. Add the \'overdue\' column from `stg_orders` to the `orders` model.\n    \n  - **Validation Status for Each Change:**\n    - Change 1: **Correct**. The \'overdue\' column from `stg_orders` is directly integrated into the `orders` model without calculations.\n    \n  - **Sections of Code Needed to Change:**\n    ```sql\n    WITH orders AS (\n      SELECT *,\n             overdue  -- Add the overdue column\n      FROM {{ ref(\'stg_orders\') }}  -- Source model\n    ),\n    ```\n\n  - **Documentation Changes:**\n    ```yaml\n    models:\n      - name: orders\n        descr

### Test local model

In [26]:
from crewai import LLM, Agent, Task, Crew

local_llm = LLM(model="lm_studio/Llama-3.2-3B-Instruct-4bit", base_url="http://127.0.0.1:1234/v1", api_key="asdf")

In [27]:
agent = Agent(
    role="Data Analyst",
    goal="Analyze eCommerce sales data",
    backstory="Expert in data analytics with years of experience",
    llm=local_llm
)

task = Task(
    description="Analyze sales trends from the last quarter and identify key insights.",
    agent=agent,
    expected_output="A detailed report summarizing sales trends, key insights, and recommendations."
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff() 
print(result)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable


# Agent: Data Analyst
## Task: Analyze sales trends from the last quarter and identify key insights.


# Agent: Data Analyst
## Final Answer: 
**Sales Trends Report for the Last Quarter**

The sales trends analysis reveals several key insights and recommendations.

**Top-Performing Product Categories:**
Based on sales data, the top-performing product categories for the last quarter are:

1. Electronics (23% of total sales)
2. Clothing and Accessories (20% of total sales)
3. Home and Kitchen (18% of total sales)

These categories are expected to continue performing well in the coming quarters. Recommendations:

- Continue investing in product offerings and marketing strategies for these categories
- Consider expanding into new markets or geographic locations with high demand for these products

**Geographic Location Analysis:**
Based on sales data, the top-performing geographic locations for the last quarter are:

1. North America (30% of total sales)
2. Europe (25% of total sales)
3. A

In [28]:
local_flow = dbtChatFlow(custom_llm = local_llm)
local_flow.plot()

user_input = "I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it "
result = local_flow.kickoff(inputs={"request": user_input, "dbt_repo_knowledge_df": dbt_repo_knowledge_df, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})
display(Markdown(f"<div style='font-size: 18px;'><b>User input:</b> <i>{user_input}</i></div><hr>"))
display(Markdown(result.raw))

TypeError: dbtChatFlow.__init__() missing 1 required positional argument: 'files'